Importing Libraries

In [2]:
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import os

from zipfile import ZipFile



Extract Dataset

In [86]:
zip_file_path ="BanglaLekha-Isolated.zip"
extracted_folder = os.getcwd()
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)


Transformation

In [4]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.Grayscale(num_output_channels=1),  
    transforms.ToTensor(),
])

Image Loader

In [5]:
data_path="BanglaLekha-Isolated/Images"
dataset = ImageFolder(root=data_path, transform=transform)

Splitting Dataset

In [6]:
dataset_size = len(dataset)
train_size = int(0.99 * dataset_size)
dev_size = test_size = (dataset_size - train_size) // 2

Dataloaders

In [7]:
train_set, dev_set, test_set = random_split(dataset, [train_size, dev_size, test_size])
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [8]:
for loader, set_name in zip([train_loader, dev_loader, test_loader], ["Training", "Dev", "Test"]):
    inputs, labels = next(iter(loader))
    
    # Remove the singleton dimension (channel dimension) using squeeze
    inputs = inputs.squeeze(1)
    
    print(f"{set_name} set:")
    print("Batch shape:", inputs.shape)
    print("Labels:", labels)
    print("=" * 30)

Training set:
Batch shape: torch.Size([32, 32, 32])
Labels: tensor([63, 11, 71, 60, 18, 30, 20, 65, 59, 37, 29, 22, 71, 81, 30,  3, 31, 29,
        23, 64,  9,  1, 48, 38, 39, 81, 49, 18, 69, 19,  4, 39])
Dev set:
Batch shape: torch.Size([32, 32, 32])
Labels: tensor([67, 40, 60, 78,  8, 13, 29, 60, 12, 30,  9, 38, 74, 49, 65, 46, 54, 19,
        63,  5,  9, 42, 83, 66, 10, 13,  9, 51, 72, 82, 22,  3])
Test set:
Batch shape: torch.Size([32, 32, 32])
Labels: tensor([58, 67, 13, 27, 73, 71, 49, 16, 10, 82, 23, 78, 79, 32, 25, 73, 40, 59,
        45, 48, 54,  0, 75, 71, 18, 45,  5, 47, 71, 32,  3, 77])


Model

In [9]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

Training Function

In [10]:
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass (compute predictions)
        outputs = model(inputs)
        # Compute the loss
        loss = criterion(outputs, labels)
        loss.backward()
        #Backpropagation
        optimizer.step()
        optimizer.zero_grad()
        # Accumulate the loss
        total_loss += loss.item()
    # Calculate average training loss
    avg_train_loss = total_loss / len(train_loader)

    return avg_train_loss


Validate Function

In [11]:
def validate_model(model, dev_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0
    total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in dev_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass (compute predictions)
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predicted class indices
            _, predicted = torch.max(outputs, 1)

            # Count correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

    # Calculate accuracy and average loss
    accuracy = total_correct / total_samples
    avg_loss = total_loss / len(dev_loader)
    return accuracy, avg_loss

Testing Function

In [12]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0
    total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass (compute predictions)
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predicted class indices
            _, predicted = torch.max(outputs, 1)

            # Count correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

    # Calculate accuracy and average loss
    test_accuracy = total_correct / total_samples
    avg_loss = total_loss / len(test_loader)
    print(f"Test Accuracy: {test_accuracy:.4f}, Average Loss: {avg_loss:.4f}")

Instance of the Model

In [13]:
num_classes = len(dataset.classes)
model = SimpleCNN(num_classes=num_classes)

Define Device

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=4096, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=84, bias=True)
  )
)

Loss Function and Optimizer

In [19]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)


Path to save and load model

In [16]:
save_path = os.path.join(os.getcwd(), "save_model.pth")
load_path = "./save_model.pth"


Number of epoch

In [17]:
num_epochs = 1

Training 

In [18]:
for epoch in range(num_epochs):
    train_model(model, train_loader, optimizer, criterion, device)

    # Validate the model
    validation_accuracy, avg_validation_loss = validate_model(model, dev_loader, criterion, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {validation_accuracy:.4f}")

    # Save the model at the specified path
    if save_path is not None:
        torch.save(model.state_dict(), save_path)
        print(f"Model saved at epoch {epoch+1}")    

KeyboardInterrupt: 

Load Model

In [112]:
loaded_model = SimpleCNN(num_classes=num_classes)
loaded_model.load_state_dict(torch.load(load_path))
loaded_model.to(device)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=200704, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=84, bias=True)
  )
)

Final Testing

In [113]:
test_model(model, test_loader, criterion, device)

Test Accuracy: 0.7292, Average Loss: 1.0113
